<a href="https://colab.research.google.com/github/cytoscape/cytoscape-automation/blob/master/for-scripters/Python/advanced-view-api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# py4cytoscape: View API Examples

## Yihang Xin and Alex Pico

## 2021-11-16


## Introduction

This is a sample Jupyter Notebook to demonstrate how users can manipulate Cytoscape network views directly from Python code.

    
#### Use Cases
* __Create your own layout algorithms__
* __Create annimations__
    

## Installation
The following chunk of code installs the `py4cytoscape` module.

In [1]:
%%capture
!python3 -m pip install python-igraph requests pandas networkx
!python3 -m pip install py4cytoscape

If you are using a remote notebook environment such as Google Colab, please execute the cell below. (If you're running on your local notebook, you don't need to do that.)



In [2]:
import requests
exec(requests.get("https://raw.githubusercontent.com/cytoscape/jupyter-bridge/master/client/p4c_init.py").text)
IPython.display.Javascript(_PY4CYTOSCAPE_BROWSER_CLIENT_JS) # Start browser client

You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


Loading Javascript client ... cce958d8-a19c-4801-a3c9-63b895e8d4c4 on https://jupyter-bridge.cytoscape.org


<IPython.core.display.Javascript object>

## Basic Setup

* Import required packages
* Load a network(s)

## Import the required packages


In [3]:
import pandas as pd
import json
import py4cytoscape as p4c

In [4]:
# Reset current session for fresh start
p4c.close_session(False)

# Load a sample network
network1 = p4c.open_session()

Opening sampleData/sessions/Yeast Perturbation.cys...


In [5]:
# You can check all available Visual Properties with this function call:
vps = p4c.get_visual_property_names()
vps

['COMPOUND_NODE_PADDING',
 'COMPOUND_NODE_SHAPE',
 'DING_RENDERING_ENGINE_ROOT',
 'EDGE',
 'EDGE_BEND',
 'EDGE_CURVED',
 'EDGE_LABEL',
 'EDGE_LABEL_COLOR',
 'EDGE_LABEL_FONT_FACE',
 'EDGE_LABEL_FONT_SIZE',
 'EDGE_LABEL_ROTATION',
 'EDGE_LABEL_TRANSPARENCY',
 'EDGE_LABEL_WIDTH',
 'EDGE_LINE_TYPE',
 'EDGE_PAINT',
 'EDGE_SELECTED',
 'EDGE_SELECTED_PAINT',
 'EDGE_SOURCE_ARROW_SELECTED_PAINT',
 'EDGE_SOURCE_ARROW_SHAPE',
 'EDGE_SOURCE_ARROW_SIZE',
 'EDGE_SOURCE_ARROW_UNSELECTED_PAINT',
 'EDGE_STACKING',
 'EDGE_STACKING_DENSITY',
 'EDGE_STROKE_SELECTED_PAINT',
 'EDGE_STROKE_UNSELECTED_PAINT',
 'EDGE_TARGET_ARROW_SELECTED_PAINT',
 'EDGE_TARGET_ARROW_SHAPE',
 'EDGE_TARGET_ARROW_SIZE',
 'EDGE_TARGET_ARROW_UNSELECTED_PAINT',
 'EDGE_TOOLTIP',
 'EDGE_TRANSPARENCY',
 'EDGE_UNSELECTED_PAINT',
 'EDGE_VISIBLE',
 'EDGE_WIDTH',
 'EDGE_Z_ORDER',
 'NETWORK',
 'NETWORK_ANNOTATION_SELECTION',
 'NETWORK_BACKGROUND_PAINT',
 'NETWORK_CENTER_X_LOCATION',
 'NETWORK_CENTER_Y_LOCATION',
 'NETWORK_CENTER_Z_LOCATION

## Get Network View

In [6]:
# Get views for a network: Cytoscape "may" have multiple views, and that's why it returns list instead of an object.
view_id_list = p4c.get_network_views()

# Display IDs of available views
print(view_id_list)

[1800841]


In [7]:
# Choose visual_property from vps
x_loc_dict = p4c.get_node_property(visual_property='NODE_X_LOCATION')
y_loc_dict = p4c.get_node_property(visual_property='NODE_Y_LOCATION')

### Convert to Pandas Data Frame
Since it's a simple dictionary, you can easily convert it into Data Frame.

In [8]:
x_loc_df = pd.DataFrame.from_dict(x_loc_dict, orient='index' )
x_loc_df.columns = ['NODE_X_LOCATION']
x_loc_df.head()

,NODE_X_LOCATION
YBR045C,1658.702983
YPR010C,3396.704051
YMR005W,1820.682109
YJL194W,2463.772197
YDL078C,1177.669830


In [9]:
y_loc_df = pd.DataFrame.from_dict(y_loc_dict, orient='index' )
y_loc_df.columns = ['NODE_Y_LOCATION']
y_loc_df.head()

,NODE_Y_LOCATION
YBR045C,844.886463
YPR010C,2113.540119
YMR005W,2267.999927
YJL194W,2621.401966
YDL078C,3331.536427


In [10]:
nodes_name = p4c.get_all_nodes()
nv1 = nodes_name[0]
print(nv1)

YHR171W


In [11]:
# Access single value
nv1_color = p4c.get_node_property(visual_property='NODE_FILL_COLOR', node_names=nv1)
print(nv1_color)

{'YHR171W': '#F1F7FC'}


## Directly set Visual Property values

Setting values are also easy with py4cytoscape.  All you have to do is creating a dictionary for the target nodes and edges.

###  Set random colors to nodes

In [12]:
# Switch current visual stye to a simple one...
p4c.create_visual_style('Minimal')
p4c.set_visual_style('Minimal', network=network1)

{'message': 'Visual Style applied.'}

In [13]:
# Change background color to black
p4c.set_background_color_default('#000000', style_name='Minimal')

''

In [14]:
target_nodes = p4c.get_node_property(visual_property='NODE_LABEL')

In [15]:
# Generate random colors for each node
import random

def get_color():
    r = random.randint(0, 255)
    g = random.randint(0, 255)
    b = random.randint(0, 255)
    return '#' + hex(r)[2:] + hex(g)[2:] + hex(b)[2:]

# Assign key-value pair.  For this example, node SUID to color.
def generate_randome_color_dict(node_ids):
    new_values = {}
    for n in node_ids:
        new_values[n] = get_color()
    return new_values

new_values = generate_randome_color_dict(target_nodes)

In [16]:
# Set new values for a set of nodes.  In this case, all nodes in the network
NODE_FILL_COLOR = p4c.map_visual_property('node fill color', 'name', 'd', list(new_values.keys()), list(new_values.values()))
p4c.update_style_mapping('Minimal', NODE_FILL_COLOR)
p4c.set_visual_style('Minimal')

{'message': 'Visual Style applied.'}